# 세그먼트 트리(Segment Tree)

* 참고 글 : https://m.blog.naver.com/ndb796/221282210534 (안경잡이 개발자)


### 세그먼트 트리(Segment Tree)

- [https://m.blog.naver.com/ndb796/221282210534](https://m.blog.naver.com/ndb796/221282210534)
- **여러 개의 데이터가 연속적으로 존재할 때, 특정한 범위의 데이터의 합을 구하는 방법**
- 비슷한 성능으로 인덱스 트리가 있으며, 인덱스 트리가 세그먼트 트리에 비해 메모리 성능이 더 좋음

### 기존의 배열을 '구간 합 트리' 로 새롭게 생성하기

- **구간 합 트리의 인덱스는 1로 시작한다.**
    - **인덱스가 1로 시작하는 것은 2를 곱했을 때, 왼쪽 자식노드를 가리키므로, 검색에 효과적이어서.**
- 구간 합 트리의 1번 노드에는 기존 인덱스 0~11까지(전체)의 값의 총합 66을 넣는다.
- 1번 노드의 자식 노드인 2, 3번 노드에는 각각 절반씩 쪼갠 범위 합을 넣는다.
    - 2번 노드 : 기존 인덱스 0~5까지의 값의 총합
    - 3번 노드 : 기존 인덱스 6~11까지의 값의 총합

In [1]:
arr = [1, 9, 3, 8, 4, 5, 5, 9, 10, 3, 4, 5]  # 원글 a
tree = [0 for i in range(len(arr)*4)] # 원글 tree, 4를 곱하면 arr의 길이 12와 가까운 16의 2배인 32보다 크므로 전 범위 커버 가능

![](seg0.png)

* init 함수
    * tree를 누적합 트리로 초기화하는 함수
    * L = 0, R = len(arr)-1, node = 1로 넣어주어 초기화
        * L : arr의 첫 인덱스
        * R : arr의 끝 인덱스
        * node : 1 (누적합 트리의 루트 노드)  
        
          
* (내 의문점) 굳이 node를 입력인자로 받을 필요가 있나?
    * 어차피 초기화 목적이라면, node는 처음에 1값만 들어가고, ... 아..
    * 재귀적 호출을 위해서 필요해 보임

In [2]:
# L = 0, R = len(arr)-1 = 11, node = 1로 넣어주면, tree는 누적합 트리로 초기화하는 함수
def init(L, R, node): 
    
    if L == R:    # 로직 1, 인덱스가 같으면, 해당 노드값 반환 tree[node] = arr[L]
        tree[node] = arr[L]    # 여기서 주의할 점 !!!. L과 node는 다른 값이다.(L은 0부터 시작하는 index이고, node는 1부터 시작하는 인덱스이므로)
        return tree[node]
    
    # 로직 2, 인덱스 L, R 이 다를 경우, tree[node] = tree[2*node] + tree[2*node+1]
    mid = (L+R) // 2
    tree[node] = init(L, mid, 2*node) + init(mid+1, R, 2*node+1)
    return tree[node]


# 예시)
#                                            init(0, 11, 1)
# =                           init(0, 5, 2)          +         init(6, 11, 3)
# =                 init(0, 2, 4)    +    init(3, 5, 5)    init(6, 8, 6) + init(9, 11, 7)
# =         init(0, 1, 8)    +    init(1, 2, 9) ...
# = init(0, 0, 16) + init(1, 1, 17) ...

![](seg1.png)

In [27]:
init(0, len(arr)-1, 1)

print(tree)

[0, 66, 30, 36, 13, 17, 24, 12, 10, 3, 12, 5, 14, 10, 7, 5, 1, 9, 0, 0, 8, 4, 0, 0, 5, 9, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


![](seg2.png)

 * Gsum : 구간 합을 구하는 함수
 
     * 누적합 트리를 보면, 아래 로직대로, node값에 따른 L, R이 정해지는 것을 알 수 있음
     * 초깃값을 L = 0, R = len(arr)-1, node=1 로 설정하면,
     * 누적합 트리에 따라 2번 노드는 0 ~ mid까지의 누적합을 나타냄

In [18]:
# L, R : 투 포인터와 비슷한 역할(탐색할 범위), 초깃값은 누적합 트리의 원래 배열 인덱스 범위(0 ~ 11)
# start, end : 구간 합에 대한 범위 인덱스

def Gsum(L, R, node, start, end):
    
    # 범위 밖에 있는 경우
    if R < start or end < L :
        return 0
    
    # 범위 안에 있는 경우
    if start <= L and R <= end:
        return tree[node]
    
    mid = (L + R) // 2
    
    return Gsum(L, mid, 2*node, start, end) + Gsum(mid+1, R, 2*node+1, start, end)

In [16]:
Gsum(0, 11, 1, 0, 11)

66

In [17]:
Gsum(0, 11, 1, 3, 8)

41

 * update : 특정 원소의 값을 수정하는 함수
     * 수정할 원소를 포함하고 있는 모든 구간 합 노드들을 갱신해줌
     * 예를 들어 인덱스 7번 값을 수정할 경우, 아래와 같은 노드 수정

![](seg3.png)

In [21]:
# index : 수정하고자 하는 값의 인덱스
# dif : 수정할 값(바뀐 값 자체가 아니라 원래 값과 변경된 값의 차이)

def update(L, R, node, index, dif):
    
    # 범위 밖에 있는 경우
    if index < L or index > R :
        return
    
    # 범위 안에 있는 경우, 내려가며, 연쇄적으로 다른 원소도 갱신
    tree[node] += dif
    
    if L == R:
        return
    
    mid = (L + R) // 2
    
    update(L, mid, 2*node, index, dif)
    update(mid+1, R, 2*node+1, index, dif)

In [32]:
def main():
    
    init(0, len(arr)-1, 1)
    
    print("0 ~ 11까지의 구간 합 : ", Gsum(0, 11, 1, 0, 11))
    print("3 ~ 8까지의 구간 합 : ", Gsum(0, 11, 1, 3, 8))
    
    print()
    print(" * 수정 전 tree")
    print(tree)
    print()
    
    print("* 인덱스 7을 +2 만큼 수정 후 tree")
    update(0, 11, 1, 7, 2)
    print(tree)

In [33]:
main()

0 ~ 11까지의 구간 합 :  66
3 ~ 8까지의 구간 합 :  41

 * 수정 전 tree
[0, 66, 30, 36, 13, 17, 24, 12, 10, 3, 12, 5, 14, 10, 7, 5, 1, 9, 0, 0, 8, 4, 0, 0, 5, 9, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

* 인덱스 7을 +2 만큼 수정 후 tree
[0, 68, 30, 38, 13, 17, 26, 12, 10, 3, 12, 5, 16, 10, 7, 5, 1, 9, 0, 0, 8, 4, 0, 0, 5, 11, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Lazy Propagation

- 참고 글
    - [https://ojt90902.tistory.com/562](https://ojt90902.tistory.com/562)
    - [https://baeharam.github.io/posts/algorithm/lazy-propagation/]
    
### 세그먼트 트리의 한계

- 세그먼트 트리는 구간 합을 구하는데, O(logN)이 필요
- 그러나, 구간의 값을 업데이트하는 상황이면,
    - 단일 노드 수정시, O(logN)
    - 구간(n개 범위)의 노드 수정시, O(nlogN)
        - 업데이트가 m번 수행된다면, O(nmlogN)
        
### Lazy propagation은 m번의 쿼리가 들어왔을 때, O(mlogN)의 시간 복잡도를 가짐  

# Lazy Propagation 구현

### Lazy Propagation을 위해서 기본 세그먼트 트리와 달라지는 점은 크게 2가지

1. 트리 초기화 시에 리스트 형식으로 각 노드를 구성해 Lazy 값을 저장할 수 있게 한다.
2. 업데이트, 쿼리 시에 함수 형식이 조금 바뀐다.

### init() 함수 : 트리 초기화
* tree는 각 요소로 길이가 2인 리스트를 가짐
    * 0번 인덱스에는 값이 저장되고, 
    * 1번 인덱스에는 Lazy가 저장됨

In [1]:
# 재귀 형식으로 tree 초기화
# arr : 값이 저장되어 있는 배열
# L, R은 각각 arr의 처음과 마지막 인덱스 (0, len(arr)-1)
# node는 1로 초기화
# tree는 각 요소로 길이가 2인 리스트를 가짐
# tree를 입력인자로 지정했으므로, 함수안에서 따로 초기화 할 필요는 없음

# 사실상 tree를 입력인자로 받는 부분과, 값을 tree[node][0]에 받는 부분 빼고는 세그먼트 트리의 init과 동일함

def init(L, R, node, tree):
    
    if L == R:
        tree[node][0] = arr[L]
        return tree[node][0]
    
    else:
        mid = (L + R) // 2
        tree[node][0] = init(L, mid, 2*node, tree) + init(mid+1, R, 2*node+1, tree)
        return tree[node][0]

### Lazy Propagation 구현 (propagation 함수)
* 상위 노드의 Lazy값을 자식 노드에게 전달하고, 자신의 노드에 적용하는 함수

* 세그먼트 트리와 달리진점
    * 노드에 Lazy값이 있는치 확인하고 처리하는 부분 추가됨
        * 현재 잎새 노드라면, 
            1. Lazy값을 현재 노드에 반영해주고, 
            2. Lazy값을 0으로 초기화
        * 현재 잎새 노드가 아니라면,
            1. Lazy값을 자식들에게 전달해주고,
            2. 현재 노드에 Lazy값을 반영해주고, 
            3. 그 다음 함수를 진행

In [ ]:
def propagation(L, R, node, tree):
    
    if L != R:    # 잎새노드가 아니면, Lazy값 자식에게 전달
        tree[2*node][1] += tree[node][1]
        tree[2*node+1][1] += tree[node][1]
        
    # 현재 노드에 Lazy값 *(범위) 만큼 전달 후, 초기화
    tree[node][0] += tree[node][1] * (R-L+1)
    tree[node][1] = 0
    
    return

### 연속 구간 업데이트(modify 함수)

    * def modify(L, R, node, tree, start, end, value)
        * start, end : 업데이트 할 구간
        * value : 업데이트 할 값

In [ ]:
def modify(L, R, node, tree, start, end, value):
    
    # line 4~11 없어도 되지 않나???
    if L == R and L == start:    # 업데이트 구간의 잎새 노드이면,
        # 나는 위애 조건 중 L == start가 "start <= L and L <= end" 로 바뀌어야 할것 같다고 생각함
        if tree[node][1] != 0:   # 현재 노드에 Lazy값이 있으면
            tree[node][0] += tree[node][1]   # 현재 노드의 Lazy값 더하고 초기화
            tree[node][1] = 0
            
        tree[node][0] += value    # 현재노드에 값 업데이트 및 반환
        return tree[node][0]
    
    else:
        if tree[node][1] != 0:    # 현재 노드에 Lazy값이 있으면, 전파 수행
            propagation(L, R, node, tree)
            
        # 수정 범위 내에 있을 떄,
        if start <= L and R <= end:
            tree[node][0] += value * (R-L+1)    # 현재 노드 수정
            
            if L != R :  # 현재 노드가 자식 노드를 갖는다면, 
                tree[2*node][1] += value    # 자식 노드의 Lazy에 value 만큼 추가
                tree[2*node+1][1] += value
                # 이 자식 노드들도 자식이 몇개 있는지 확인해서 그만큼 곱해야하지 않나????
                # 아닌듯, Lazy값이 해당 노드에 저장되면, 어차피 propagation함수에서 Lazy를 처리 할 때, 자식 갯수 만큼 곱해져서               
            return tree[node][0]
        
        # 수정 범위 밖에 있을 때, 
        elif R < start or end < L:
            return tree[node][0]
        
        else:  # 수정 범위가 탐색 범위와 일정부분 겹쳐저 있을 때, 쪼개서 다시 수행
            mid = (L+R)//2
            a = modify(L, mid, 2*node, tree, start, end, value)
            b = modify(mid+1, R, 2*node+1, tree, start, end, value)
            tree[node][0] + a + b
            return tree[node][0]
    

* 내 생각대로 modify 수정

In [ ]:
def modify(L, R, node, tree, start, end, value):
    
    # 현재노드가 수정이 되든 안되든 Lazy값은 적용이 된 후, return 되어야함
    if tree[node][1] != 0:    # 현재 노드에 Lazy값이 있으면, 전파 수행
        propagation(L, R, node, tree)
    
    # 탐색 범위가 수정 범위 밖에 있을 때, 
    if R < start or end < L:
        return tree[node][0]
    
    else:           
        # 수정 범위 내에 있을 떄,
        if start <= L and R <= end:
            tree[node][0] += value * (R-L+1)    # 현재 노드 수정
            
            if L != R :  # 현재 노드가 자식 노드를 갖는다면, 
                tree[2*node][1] += value    # 자식 노드의 Lazy에 value 만큼 추가
                tree[2*node+1][1] += value
                # 이 자식 노드들도 자식이 몇개 있는지 확인해서 그만큼 곱해야하지 않나????
                # 아닌듯, Lazy값이 해당 노드에 저장되면, 어차피 propagation함수에서 Lazy를 처리 할 때, 자식 갯수 만큼 곱해져서               
            return tree[node][0]
        
        else:  # 수정 범위가 탐색 범위와 일정부분 겹쳐저 있을 때, 쪼개서 다시 수행
            mid = (L+R)//2
            a = modify(L, mid, 2*node, tree, start, end, value)
            b = modify(mid+1, R, 2*node+1, tree, start, end, value)
            tree[node][0] + a + b
            return tree[node][0]
    